In [1]:
"""
MPhys Project Semester 1
Mila Jelic & Paul Vautravers
"""

import numpy as np
import json
import tensorflow as tf
import gc
import os
import gen_fbm_nn_model as fbm_nn

In [2]:
def get_disp_dict_from_file(filename, window, step_size, max_step_size, restriction):

    with open('disp_dict_data/'+filename+'_s{}/'.format(max_step_size)+'disp_dict_'+filename
              +'_w{}_step{}_r{}'.format(window, step_size, restriction), 'r') as f:
        disp_dict = f.read()
    
    return json.loads(disp_dict)

def load_nn_model(window_size,n_samples=10000,n_epochs=100):

    try:
        model = tf.keras.models.load_model("model3dense_n{}.h5".format(window_size))
    except OSError:
        fbm_nn.__main__(n_samples,window_size,n_epochs)
        model = tf.keras.models.load_model("model3dense_n{}.h5".format(window_size))
    return model 

def estimate_hurst(disp_dict, window):
    # for one step size
    
    model = load_nn_model(window)
    
    half_window = int(window/2)
    
    if window % 2 == 1:  # odd window size
        stop_index = 2+half_window
    else:  # even window size
        stop_index = 1+half_window
    
    h = np.array([])
    
    keys_list = list(disp_dict)  # keys are 'TrackID[downsampling start index]'
    length = len(keys_list)
    for i, ds_tid in enumerate(keys_list):
        print('Track: {}   ({}/{})'.format(ds_tid, i+1, length))
        disps = np.array(disp_dict[ds_tid])
        for j in range(half_window, len(disps)-(1+half_window)):
            inx = disps[(j-half_window):(j+stop_index)]
            inx = np.array([(inx[1:]-inx[0:-1])/(np.amax(inx)-np.amin(inx))])
            test = model.predict(inx, verbose=0)
            h = np.append(h, test[0][0])
    return h

# def estimate_hurst(disp_dict, window):
#     # for one step size
    
#     model = load_nn_model(window)
    
#     half_window = int(window/2)
    
#     if window % 2 == 1:  # odd window size
#         stop_index = 2+half_window
#     else:  # even window size
#         stop_index = 1+half_window
    
#     h = np.array([])
    
#     keys_list = list(disp_dict)  # keys are track IDs
#     length = len(keys_list)
#     for i, ds_tid in enumerate(keys_list):
#         print('Track: {}   ({}/{})'.format(ds_tid, i+1, length))
#         disps = np.array(disp_dict[ds_tid])
#         inx_arr = [disps[(i-half_window):(i+stop_index)] for i in range(half_window, len(disps)-(1+half_window))]
#         inx_diff = [np.array([(inx[1:]-inx[0:-1])/(np.amax(inx)-np.amin(inx))]) for inx in inx_arr]
#         h_test = [model.predict(inx,verbose=0)[0][0] for inx in inx_diff]
#         h = np.append(h, h_test)
#     return h

def save_h_data_single_step(h_data_single_step, filename, window, step_size, max_step_size, restriction):
    # save h data for one step size (and all tracks) to csv file
    
    path = 'h_data/'+filename+'_s{}/'.format(max_step_size)
    try:
        np.savetxt(path+'h_data_'+filename+'_w{}_step{}_r{}'.format(window, step_size, restriction), 
                   h_data_single_step, delimiter=',')
    except FileNotFoundError:
        os.makedirs(path)
        np.savetxt(path+'h_data_'+filename+'_w{}_step{}_r{}'.format(window, step_size, restriction), 
                   h_data_single_step, delimiter=',')

#def main(filenames, window, step_sizes, restriction):

#    for f in filenames:
#        s = 1
#        while s<=max(step_sizes):
#            print('Step size: {}'.format(s))
#            disp_dict = get_disp_dict_from_file(f, window, s, max(step_sizes), restriction)
#            h_data = estimate_hurst(disp_dict, window)
#            save_h_data_single_step(h_data, f, window, s, max(step_sizes), restriction)
#            del h_data
#            gc.collect()
#            s+=1

def main(filenames, window, step_sizes, restriction):

    for f in filenames:
        for s in step_sizes:
            print('Step size: {}'.format(s))
            disp_dict = get_disp_dict_from_file(f, window, s, max(step_sizes), restriction)
            h_data = estimate_hurst(disp_dict, window)
            save_h_data_single_step(h_data, f, window, s, max(step_sizes), restriction)
            del h_data
            gc.collect()

In [3]:
filenames = np.array(['Control_frame001-200',
                      'Control_frame200-400',
                      'Control_frame400-600',
                      'Control_frame600-800',
                      'Control_frame800-1000',
                      'Control_frame1000-1200',
                      'LanB1_frame001-200',
                      'LanB1_frame200-400',
                      'LanB1_frame400-600',
                      'LanB1_frame600-800',
                      'LanB1_frame800-1000',
                      'LanB1_frame1000-1200',
                      'defLanB1_300817_frame200-400',
                      'defLanB1_300817_frame400-600'])
window = 20
step_sizes = [1,2,3,4,5,6,7,8]
restriction = 10000

In [ ]:
main(filenames[0:1], window, step_sizes, restriction)

# can be slow, may need to do one file at a time

Step size: 1
Track: 1000000060[0]   (1/25)
Track: 1000000065[0]   (2/25)
Track: 1000000070[0]   (3/25)
Track: 1000000083[0]   (4/25)
Track: 1000000094[0]   (5/25)
Track: 1000000105[0]   (6/25)
Track: 1000000121[0]   (7/25)
Track: 1000000122[0]   (8/25)
Track: 1000000132[0]   (9/25)
Track: 1000000136[0]   (10/25)
Track: 1000000143[0]   (11/25)
Track: 1000000145[0]   (12/25)
Track: 1000000150[0]   (13/25)
Track: 1000000151[0]   (14/25)
Track: 1000000161[0]   (15/25)
Track: 1000000162[0]   (16/25)
Track: 1000000168[0]   (17/25)
Track: 1000000177[0]   (18/25)
Track: 1000000409[0]   (19/25)
Track: 1000076605[0]   (20/25)
Track: 1000077211[0]   (21/25)
Track: 1000077755[0]   (22/25)
Track: 1000078698[0]   (23/25)
Track: 1000079023[0]   (24/25)
Track: 1000080241[0]   (25/25)
Step size: 2
Track: 1000000060[0]   (1/50)
Track: 1000000060[1]   (2/50)
Track: 1000000065[0]   (3/50)
Track: 1000000065[1]   (4/50)
Track: 1000000070[0]   (5/50)
Track: 1000000070[1]   (6/50)
Track: 1000000083[0]   (7/50